In [5]:
import json
import numpy as np
import pretty_midi

In [6]:
from ExtractData import *

In [7]:
path = '../Data/DoodleSample'

In [8]:
all_phrases = get_cleaned_phrases(path, 3)

In [30]:
def pitch_and_length(phrase, part):
    """
    A function that takes the array and splits a part into lists of note pitch and length.
    
    Args:
        - phrase (tuple): a tuple of lists of the form (pitches, note_start_step, note_end_step, instruments).
        - part (int): an integer between 0 and 3 (inclusive) to denote which of the four parts.
        
    Returns:
        - Two lists for the note pitch and length.
    """
    # Extract a single part of the phrase
    phrase_part = phrase[part].tolist()
    
    # Initialize empty lists for note lengths and unique pitches
    note_lengths = []
    pitches = []

    # Initialize variables to track the current note and its length
    current_note = None
    current_length = 0

    # Iterate through the original list
    for pitch in phrase_part:
        if pitch == current_note:
            # Increment the length of the current note
            current_length += 1
        else:
            if current_note is not None:
                # Append the length of the previous note
                note_lengths.append(current_length)
                # Append the unique pitch
                pitches.append(current_note)

            # Start counting a new note
            current_note = pitch
            current_length = 1

    # Append the last note's length and unique pitch
    note_lengths.append(current_length)
    pitches.append(current_note)
    
    # Divide the length by two to represent the number of beats
    note_lengths = [note_length/2 for note_length in note_lengths]
    
    return pitches, note_lengths

In [33]:
sample_phrase = all_phrases[0]
pitch_and_length(sample_phrase, 0)

([71, 69, 74, 72, 71, 67, 71, 69, 71, 69],
 [1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 4.0])

In [71]:
all_phrases[0]

array([[71, 71, 69, 69, 74, 74, 74, 74, 72, 72, 71, 71, 67, 67, 71, 71,
        69, 69, 69, 69, 71, 71, 71, 71, 69, 69, 69, 69, 69, 69, 69, 69],
       [67, 67, 67, 67, 69, 69, 69, 69, 67, 67, 66, 66, 67, 67, 67, 64,
        64, 64, 66, 66, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67],
       [62, 62, 62, 62, 62, 62, 62, 62, 64, 64, 62, 62, 67, 67, 55, 67,
        57, 57, 57, 57, 62, 62, 62, 62, 64, 64, 64, 64, 57, 57, 62, 62],
       [43, 43, 43, 43, 55, 55, 54, 54, 52, 52, 50, 50, 50, 52, 50, 50,
        48, 48, 48, 48, 47, 47, 47, 47, 49, 49, 45, 45, 50, 50, 50, 50]])

In [89]:
class PhraseToMIDI:
    def __init__(self, phrase, tempo=120):
        """
        Initialize the PhraseToMIDI object.

        Args:
        phrase (list): A list of dictionaries, each containing 'pitch_list' and 'length_list' for a part.
        tempo (int): Optional. The tempo (BPM) for the MIDI file. Default is 120.
        """
        self.phrase = phrase
        self.tempo = tempo
        
    def write_phrase_to_MIDI(self, phrase_array):
        """
        A function that takes the phrase and writes it to a MIDI file.
        
        Args:
            - phrase (list): An array containing the pitches of the four parts.
            
        Returns:
            - A MIDI file storing the note data.
        """
        # Create a new MIDI data object
        midi_data = pretty_midi.PrettyMIDI(initial_tempo=self.tempo)
        
        # Name the parts
        instrument_names = ["Part 1", "Part 2", "Part 3", "Part 4"]
        
        for part in range(4):
            # Define the instrument to acoustic grand piano
            instrument = pretty_midi.Instrument(program=0, name=instrument_names[part])
            # Get the pitch and length lists
            part_pitch_list, part_pitch_length = pitch_and_length(phrase_array, part)
            for i in range(len(part_pitch_list)):
                note_start = sum(part_pitch_length[:i])
                note_end = sum(part_pitch_length[:i+1])
                # Create a Note object 
                note = pretty_midi.Note(
                    velocity=100,  
                    pitch=part_pitch_list[i],  
                    start=note_start,      
                    end=note_end         
                )
                # Add the Note object to the Instrument
                instrument.notes.append(note)
            # Add the Instrument to the MIDI data
            midi_data.instruments.append(instrument)
                
        return midi_data

In [90]:
# Create an instance of the PhraseToMIDI class
pm = PhraseToMIDI(sample_phrase, tempo=120)

# Call the write_phrase_to_MIDI method to generate MIDI data
midi_data = pm.write_phrase_to_MIDI(sample_phrase)

In [91]:
# Save the generated MIDI data to a file
midi_data.write('output.mid')

In [92]:
# Load a MIDI file
midi_data = pretty_midi.PrettyMIDI('output.mid')  # Replace with your MIDI file's path

# Iterate through the instruments in the MIDI file
for instrument in midi_data.instruments:
    print(f"Instrument: {instrument.name}")

    # Iterate through the notes in the instrument
    for note in instrument.notes[:2]:
        print(f"Note Pitch: {note.pitch}")
        print(f"Start Time: {note.start}")
        print(f"End Time: {note.end}")
        print(f"Velocity: {note.velocity}")
        print("---")


Instrument: Part 1
Note Pitch: 71
Start Time: 0.0
End Time: 1.0
Velocity: 100
---
Note Pitch: 69
Start Time: 1.0
End Time: 2.0
Velocity: 100
---
Instrument: Part 2
Note Pitch: 67
Start Time: 0.0
End Time: 2.0
Velocity: 100
---
Note Pitch: 69
Start Time: 2.0
End Time: 4.0
Velocity: 100
---
Instrument: Part 3
Note Pitch: 62
Start Time: 0.0
End Time: 4.0
Velocity: 100
---
Note Pitch: 64
Start Time: 4.0
End Time: 5.0
Velocity: 100
---
Instrument: Part 4
Note Pitch: 43
Start Time: 0.0
End Time: 2.0
Velocity: 100
---
Note Pitch: 55
Start Time: 2.0
End Time: 3.0
Velocity: 100
---
